# Comparison

In this notebook we compare composite maps and optimal projection patterns for the PlaSim dataset

In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib widget
matplotlib.rc('font', size=18)
default_colors = plt.rcParams['axes.prop_cycle'].by_key()['color']


import pandas as pd
import xarray as xr

from tqdm.notebook import tqdm

import sys
sys.path.append('../../../Climate-Learning/')

import general_purpose.utilities as ut
import general_purpose.cartopy_plots as cplt
import general_purpose.uplotlib as uplt
import general_purpose.tables as tbl

# log to stdout
import logging
logging.getLogger().level = logging.INFO
logging.getLogger().handlers = [logging.StreamHandler(sys.stdout)]
ut.indentation_sep = '  '

HOME = '../../'

In [ ]:
lon = np.load('../../lon.npy')
lat = np.load('../../lat.npy')
LON, LAT = np.meshgrid(lon, lat)

## Figure 7

In [ ]:
mask = np.load('../mask.npy')
reshaper = ut.Reshaper(mask)
reshaper.surviving_coords

### Composites

In [ ]:
ds = xr.open_dataset('../composites/composite_maps_percent5_y8000.nc')
ds

In [ ]:
T = 14
taus = [0,5,10]

In [ ]:
ga = ds.gaussian_composite.sel(T=T,tau=taus).data.reshape(len(taus),22,128,3)
ga.shape

In [ ]:
for i,tau in enumerate(taus):
    axs = cplt.mfp(LON,LAT, ga[i,:], one_fig_layout=130,figsize=(15,5),mx = 0.80,fig_num=8+np.random.randint(0,50),titles=['Temperature', 'Geopotential', 'Soil moisture'], put_colorbar=False)
    fig = axs[0].get_figure()
    # fig.suptitle(fr'$\tau = {tau}$')
    # fig.savefig(f'{HOME}C_tau{tau}.pdf')

In [ ]:
plt.close(1)
fig = plt.figure(figsize=(10,1.1), num=1)
for i in range(3):
    ax = fig.add_subplot(131 + i)
    plt.colorbar(axs[i], cax=ax, orientation='horizontal', extend='both')
    # plt.xticks(rotation=45)
    
fig.tight_layout()
# fig.savefig(f'{HOME}/C_colorbar.pdf')

### Projection patterns

In [ ]:
Ms = xr.open_dataarray('../committor/projection_patterns_T14_y8000_epsilon1_fold0.nc')
Ms

In [ ]:
for i,tau in enumerate([0,5,10]):
    proj = Ms.sel(tau=tau).data.squeeze()
    axs = cplt.mfp(LON,LAT,reshaper.inv_reshape(proj), one_fig_layout=130, figsize=(12,5), fig_num=8+i,
                   titles = None,
                   mx = [6,2,20],
                   put_colorbar=False,
                  )
    fig = axs[0].get_figure()
    # fig.suptitle(fr'$\tau = {tau}$')
    # fig.savefig(f'{HOME}/M_tau{tau}.pdf')


In [ ]:
plt.close(1)
fig = plt.figure(figsize=(10,1.1), num=1)
for i in range(3):
    ax = fig.add_subplot(131 + i)
    plt.colorbar(axs[i], cax=ax, orientation='horizontal', extend='both')
    # plt.xticks(rotation=45)
    
fig.tight_layout()
# fig.savefig(f'{HOME}/M_colorbar.pdf')